In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

import requests

# Method 2: Based on book value growth
This approach may works better for growth company that spends a lot on R&D such as Amazon, whose free cash flow is negative most of the time and thus can't use discounted cash flow to calculate its intrinsic value.

You can choose the BookValueGrowthRate of the last x years depends which one do you think is more appropriate. A commonly used time frame is 5 years or 10 years.

FutureBookValue = CurrentBookValuePerShare * (1 + BookValueGrowthRate) ^ NumberOfYears =

CurrentIntrinsicValue = FutureBookValue / ((1 + TenYearTreasuryRate) ^ NumberOfYears))  

In [2]:
def calculate_intrinsic_value_based_on_book_value_growth(
        current_book_value_per_share,
        book_value_growth_rate,
        years_to_project,
        ten_year_treasury_rate
    ):
        """
        Args:
        current_book_value_per_share:
            The current book value per share of the company.
        book_value_growth_rate:
            The average book value growth rare of the company for the last x years.
        num_of_years:
            Number of years in the future that you want to calculate based on.
        ten_year_treasury_rate:
            The 10-year treaseury rate.

        Returns:
            The calculated intrinsic value.
        """
        # Calculate future book value based on the growth rate
        future_book_value = current_book_value_per_share * (1 + book_value_growth_rate) ** years_to_project

        # Discount the future book value to get the intrinsic value
        current_intrinsic_value = future_book_value / (1 + ten_year_treasury_rate) ** years_to_project

        return current_intrinsic_value

In [3]:
# Define the function to retrieve company data
def get_company_overview(api_key, symbol):
    base_url = 'https://www.alphavantage.co/query'
    function = 'OVERVIEW'
    outputsize='compact'

    # Make the API request
    params = {
        'function': function,
        'symbol': symbol,
        'outputsize': outputsize,
        'apikey': api_key,
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [4]:
# Define the function to retrieve company balance sheet
def get_company_balance_sheet(api_key, symbol):
    base_url = 'https://www.alphavantage.co/query'
    function = 'BALANCE_SHEET'
    outputsize='compact'

    # Make the API request
    params = {
        'function': function,
        'symbol': symbol,
        'outputsize': outputsize,
        'apikey': api_key,
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [5]:
# Get the current year
current_year = datetime.today().year

# Replace 'YOUR_API_KEY' with your actual Alpha Vantage API key
api_key = '03775CCKZ199ZUG2'

# Define the symbol or stock you want to analyze
ticker = 'TSLA'  # Replace with the stock symbol you are interested in

# Call the function to get company overview data
company_overview = get_company_overview(api_key, ticker)

In [6]:
# Current book value per share

current_year = datetime.today().year

# Retrieve toal shareholders equity info
balance_sheet = get_company_balance_sheet(api_key, ticker)
total_equity = float(balance_sheet['annualReports'][0]['totalShareholderEquity'])

# Replace with the number of shares outstanding for the company
shares_outstanding = float(company_overview['SharesOutstanding'])

current_book_value_per_share = total_equity / shares_outstanding

current_book_value_per_share

14.06263762535704

In [7]:
# Book value growth rate

book_value_per_share_last1 = float(balance_sheet['annualReports'][1]['totalShareholderEquity']) / shares_outstanding
book_value_per_share_last2 = float(balance_sheet['annualReports'][2]['totalShareholderEquity']) / shares_outstanding

book_value_growth_rate_lag1 = current_book_value_per_share / book_value_per_share_last1
book_value_growth_rate_lag2 = book_value_per_share_last1 / book_value_per_share_last2

book_value_growth_rate = (book_value_growth_rate_lag1 * 0.7 + book_value_growth_rate_lag2 * 0.3) - 1

book_value_growth_rate

0.4440635489479643

In [8]:
# Num of years
years_to_project = 5

In [9]:
# Ten year treasury rate
treasury_yield10 = yf.Ticker("^TNX")
ten_year_treasury_rate = round(treasury_yield10.history(period="1d")["Close"][0] / 100, 5)

ten_year_treasury_rate

0.03928

In [10]:
# Calculate intrinsic value
intrinsic_value_bvg = calculate_intrinsic_value_based_on_book_value_growth(
        current_book_value_per_share,
        book_value_growth_rate,
        years_to_project,
        ten_year_treasury_rate)

print("{}'s intrinsic value at the end of {} based on BVG is {:.2f}".format(ticker, current_year + years_to_project, intrinsic_value_bvg))

TSLA's intrinsic value at the end of 2028 based on BVG is 72.83
